# Trusted zone

In [1]:
import duckdb
import pandas as pd

In [2]:
def df_to_DBtable(df,DB,table): # repeated
    """
    Creates a persistent table in DuckDB from the DataFrame content.
    """
    con = duckdb.connect(DB)
    con.register(table, df)
    con.execute(f'CREATE TABLE {table} AS SELECT * FROM {table}')
    con.close()

def DBtable_to_df(DB,table): # repeated
    """
    Converts the DB `table` in a data frame format 
    """
    con = duckdb.connect(DB)
    df = con.execute(f'SELECT * FROM {table}').df()
    con.close()
    return df

def get_tables(DB): # repeated
    """
    Gets all the tables from the `DB`
    """
    con = duckdb.connect(DB)
    tables = con.execute(f'SELECT table_name FROM information_schema.tables').df()['table_name']
    con.close()
    return tables

def pre_join(DB):
    """
    Adds a version (year) column to all `DB` tables in order to do the joining.
    Returns all the `DB` tables in data frame format.
    """
    tablesDB = get_tables(DB)
    dfs = []
    for table in tablesDB:
        year = ''.join(filter(str.isnumeric,table))
        df = DBtable_to_df(DB,table)
        df['Year'] = year
        dfs.append(df)
    return dfs

def trusted_zone(DB,table):
    """
    Joins all the `DB` tables into one and stores it in the `DB`.
    """
    dfs = pre_join(DB)
    df = pd.concat(dfs, axis=0, ignore_index=True)
    df_to_DBtable(df,DB,table)

In [3]:
trusted_zone('../nationalities.duckdb', 'nationalities')
trusted_zone('../household.duckdb', 'household')

### Test

#### nationalities 

In [6]:
# nationalities joined
DBtable_to_df('../nationalities.duckdb','nationalities')

,Madrid_section,Habitantes,Españoles,Extranjeros,Alemania,Austria,Bélgica,Bulgaria,Chipre,Croacia,...,Papúa_Nueva_Guinea,Timor_Oriental,Vanuatu,Otros_Países_de_Oceanía,Year,Belarús,Mónaco,Santa_Sede,República_Democrática_del_Congo,Samoa
0,0014,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
1,001401,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
2,001401001,65.0,64.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
3,0029,4559.0,3953.0,606.0,8.0,0.0,1.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
4,002901,4559.0,3953.0,606.0,8.0,0.0,1.0,13.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2018,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14518,182101,253.0,226.0,27.0,0.0,0.0,1.0,5.0,0.0,0.0,...,0.0,0.0,NaN,0.0,2019,0.0,0.0,0.0,NaN,0.0
14519,182101001,253.0,226.0,27.0,0.0,0.0,1.0,5.0,0.0,0.0,...,0.0,0.0,NaN,0.0,2019,0.0,0.0,0.0,NaN,0.0
14520,1837,1658.0,1413.0,245.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,NaN,0.0,2019,0.0,0.0,0.0,NaN,0.0
14521,183701,1658.0,1413.0,245.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,NaN,0.0,2019,0.0,0.0,0.0,NaN,0.0


#### household

In [5]:
# households joined
DBtable_to_df('../household.duckdb','household')

,section,population,single_women_aged_16_to_64,single_men_aged_16_to_64,single_women_aged_65_or_over,single_men_aged_65_or_over,adult_women_with_one_or_more_minors,adult_men_with_one_or_more_minors,two_adults_from_16_to_64_and_without_minors,two_adults_one_at_least_65_and_without_minors,...,two_adults_over_35_and_one_adult_from_16_to_34_and_two_minors,three_adults_and_0_or_more_minors,two_adults_over_35_and_two_adults_from_16_to_34,two_adults_over_35_and_two_adults_from_16_to_34_and_one_minor,two_adults_over_35_and_two_adults_from_16_to_34_and_two_or_more_minors,four_adults_and_0_or_more_minors,five_adults_and_0_or_more_minors,fifteen_or_more_inhabitants,only_minors,Year
0,1001.0,542.0,72.0,86.0,63.0,16.0,10.0,4.0,82.0,63.0,...,0.0,30.0,13.0,2.0,2.0,13.0,18.0,0.0,0.0,2018
1,1002.0,481.0,100.0,91.0,36.0,19.0,7.0,0.0,89.0,28.0,...,1.0,38.0,9.0,1.0,1.0,10.0,11.0,0.0,1.0,2018
2,1003.0,884.0,154.0,179.0,74.0,40.0,13.0,2.0,141.0,77.0,...,3.0,54.0,18.0,4.0,0.0,23.0,30.0,0.0,1.0,2018
3,1004.0,591.0,92.0,95.0,53.0,17.0,10.0,2.0,100.0,64.0,...,3.0,46.0,15.0,3.0,0.0,14.0,16.0,1.0,0.0,2018
4,1006.0,885.0,140.0,179.0,77.0,38.0,14.0,1.0,167.0,78.0,...,3.0,49.0,13.0,5.0,1.0,17.0,16.0,0.0,0.0,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7324,21029.0,526.0,35.0,33.0,7.0,5.0,21.0,9.0,36.0,24.0,...,11.0,25.0,51.0,8.0,2.0,9.0,16.0,0.0,0.0,2019
7325,21030.0,563.0,33.0,59.0,26.0,8.0,16.0,4.0,70.0,49.0,...,3.0,36.0,43.0,5.0,0.0,12.0,21.0,0.0,0.0,2019
7326,21031.0,948.0,99.0,94.0,14.0,7.0,41.0,8.0,155.0,35.0,...,6.0,29.0,25.0,6.0,0.0,17.0,23.0,0.0,0.0,2019
7327,21032.0,658.0,72.0,67.0,18.0,6.0,33.0,13.0,94.0,25.0,...,3.0,22.0,27.0,7.0,0.0,6.0,9.0,1.0,2.0,2019
